# **Langchain RAG with chat**

RAG made from the previous exemple

In [1]:
# import
from dotenv import load_dotenv
from typing import Literal
from typing_extensions import List, TypedDict
from typing_extensions import Annotated

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from langgraph.graph import START, StateGraph, MessagesState, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


In [56]:
load_dotenv()
# file_path = "/Users/mathieumuty/Repository/test/ai-test/dev/datasets/docker_cheatsheet_cli.pdf"
file_path = "/Users/mathieumuty/Repository/ai-workspace/dev/datasets/la-personne-transforme.pdf"
# file_path = "/Users/mathieumuty/Repository/test/ai-test/dev/datasets/rfc6750.pdf"

loader = PyPDFLoader(file_path)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)

all_splits = text_splitter.split_documents(docs)
ids = vector_store.add_documents(documents=all_splits)



@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

def retrieve_document_data(query: str, extend_index: int) -> Document:
    return vector_store.similarity_search(query, k=extend_index)

def run_all_check() -> None:
    print(f"document length : {len(docs)}")
    print(f"Number of chunck from text splitter : {len(all_splits)}")
    # print(retrieve_document_data('how do build a container ?', 1))
    print(retriever.batch([
        "delete a docker image?"
    ]))

run_all_check()


document length : 212
Number of chunck from text splitter : 756
[[Document(id='e0515870-0402-4a5b-b29f-bb43007af588', metadata={'producer': 'macOS Version 10.15.7 (assemblage 19H1824) Quartz PDFContext, AppendMode 1.1', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-10-04T13:42:55-04:00', 'trapped': '/False', 'moddate': "D:20220331142820Z00'00'", 'source': '/Users/mathieumuty/Repository/ai-workspace/dev/datasets/la-personne-transforme.pdf', 'total_pages': 212, 'page': 170, 'page_label': '159', 'start_index': 1537}, page_content='indiquant d’abord de quel type d’intelligence algorithmique il s’agit et comment \ncelle-ci fonctionne aujourd’hui. \nImpact réel. L ’impact réel est la prise de décision fiable, valide et rapide, \nlorsqu’il est possible de tester des systèmes d’aide à la décision efficaces : \n Apprendre à un système à reconnaître, sur une photographie aérienne \nd’une zone maritime, les différents navires présents peut servir à confronter \nl’emplacement

Part 2 : Chat 

In [57]:

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

graph_builder = StateGraph(MessagesState)

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [58]:
# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [59]:
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)


Add memory to the chat bot to keep the context of the conversation


In [60]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "hellototo"}}

## Test chat without retrieval


In [ ]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## Test Chat with retrieval

In [63]:
input_message = "Quels sont les impacts économique de l'innovation dans les entreprises qui utilisent l'AI"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Quels sont les impacts économique de l'innovation dans les entreprises qui utilisent l'AI
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_CLOVdt6b05YZXhvp4qZc0T7z)
 Call ID: call_CLOVdt6b05YZXhvp4qZc0T7z
  Args:
    query: impacts économiques de l'innovation dans les entreprises utilisant l'IA
================================= Tool Message =================================
Name: retrieve

Source: {'producer': 'macOS Version 10.15.7 (assemblage 19H1824) Quartz PDFContext, AppendMode 1.1', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-10-04T13:42:55-04:00', 'trapped': '/False', 'moddate': "D:20220331142820Z00'00'", 'source': '/Users/mathieumuty/Repository/ai-workspace/dev/datasets/la-personne-transforme.pdf', 'total_pages': 212, 'page': 127, 'page_label': '116', 'start_index': 2369}
Content: d’analyse réalisé en 2017 par la 

In [64]:
input_message = "Comment peut-on l'utiliser pour être plus productif ?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Comment peut-on l'utiliser pour être plus productif ?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_6MOXFkET9xTHpKfZARei0Vm0)
 Call ID: call_6MOXFkET9xTHpKfZARei0Vm0
  Args:
    query: comment utiliser l'IA pour améliorer la productivité
================================= Tool Message =================================
Name: retrieve

Source: {'producer': 'macOS Version 10.15.7 (assemblage 19H1824) Quartz PDFContext, AppendMode 1.1', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-10-04T13:42:55-04:00', 'trapped': '/False', 'moddate': "D:20220331142820Z00'00'", 'source': '/Users/mathieumuty/Repository/ai-workspace/dev/datasets/la-personne-transforme.pdf', 'total_pages': 212, 'page': 128, 'page_label': '117', 'start_index': 831}
Content: pourraient hausser leur rentabilité moyenne de 38 % d’ici 2035, représentant 
ainsi un fo